Axl Adilla <br>
20/466397/PPA/05963

UAS Data Science

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('gojek_twitter_dataset.csv')
dataset.head()

,tweet,sentimen
0,Saya juga mau vouchee @gojekindonesia https:/...,1
1,download gojek duluuu uwuwu,1
2,Aminnn...#orderan goride mhn di lancar kan.all...,1
3,Tq @gojekindonesia @golifeindonesia 🖤. Harusny...,1
4,Semoga Twitter panjang umur. Berkomunikasi den...,1


In [3]:
dataset['sentimen'].value_counts()

0    3062
1     938
Name: sentimen, dtype: int64

In [4]:
dataset_class_0 = dataset[dataset['sentimen'] == 0]
dataset_class_1 = dataset[dataset['sentimen'] == 1]

In [5]:
dataset_class_1_over = dataset_class_1.sample(3062, replace=True)

In [6]:
dataset_class_0_under = dataset_class_0.sample(938, replace=True)

In [7]:
dataset = pd.concat([dataset_class_0, dataset_class_1_over], axis=0)
dataset['sentimen'].value_counts()

0    3062
1    3062
Name: sentimen, dtype: int64

In [8]:
dataset_y = dataset['sentimen'].copy().values
dataset_y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [9]:
dataset_x = dataset['tweet'].copy().values
dataset_x

array(['Iyaa min akur dong :((( biar saldoku tida terpisah bagaikan adam dan hawa waktu itu :””',
       "hayu tolong dibantuin! \n@gojekindonesia\n@gojek24jam\n@infodepok_id\n@CCICPolri \nuntung mba nya nyalain data hp, jd bisa di lacak:') https://twitter.com/yuniarayutyas/status/1167387194496962560\xa0…",
       'sengaja banget nih orang... \nsumpah nyeaeeek banget gue.... \nsalah gak salah driver ttp nanggung deritanya  \n\n@gojekindonesia  pic.twitter.com/T9mmC1AlBD',
       ...,
       '@gojekindonesia kalo salah ngasi bintang ke driver karna ke pencet sama anak kecil bisa diganti ga ya? kasian soalnya kepencetnya ratingnya kecil takut berdampak negatif ke drivernya',
       'Pada akhirnya barusan GoFood mas hahahaha 😂',
       '@gojekindonesia kalo salah ngasi bintang ke driver karna ke pencet sama anak kecil bisa diganti ga ya? kasian soalnya kepencetnya ratingnya kecil takut berdampak negatif ke drivernya'],
      dtype=object)

In [10]:
import re
def remove_link(text):
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S*.com\S*', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\S*\s', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S*#\S*', '', text, flags=re.MULTILINE)
    return text

In [11]:
def remove_number_symbol(text):
    text = re.sub(r'\d.', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z]', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text, flags=re.MULTILINE)
    return text

In [12]:
import emoji
def remove_emoji(text):
    text = emoji.get_emoji_regexp().sub("", text)
    return text

In [13]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
def remove_stopwords(text):
    tokens = word_tokenize(text)
    listStopword =  set(stopwords.words('indonesian'))

    removed = []
    for t in tokens:
        if t not in listStopword:
            removed.append(t)
    return " ".join(removed)

In [14]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

sastrawi_factory = StemmerFactory()
sastrawi_stemmer = sastrawi_factory.create_stemmer()
def stemming(text):
    text = sastrawi_stemmer.stem(text)
    return text

In [15]:
def preprocessing_data(text):
    text = remove_link(text)
    text = remove_emoji(text)
    text = remove_number_symbol(text)
    text = remove_stopwords(text)
    text = stemming(text)
    
    return text

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, random_state=1, test_size=0.7)

In [17]:
def create_vector(fasttext_model, text):
    text = preprocessing_data(text)
    return fasttext_model.get_sentence_vector(text)

In [18]:
import fasttext

fasttext_model = fasttext.load_model("cc.id.100.bin")

In [19]:
x_test_vector = [ create_vector(fasttext_model, x)  for x in x_test]
x_train_vector = [ create_vector(fasttext_model, x)  for x in x_train]

In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score,cross_validate

clf = SVC()
clf.fit(x_train_vector,y_train)

SVC()

In [21]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_pred = clf.predict(x_test_vector)

classif_report = classification_report(y_test, y_pred)
confus_matrix = confusion_matrix(y_test, y_pred)

print(classif_report)
print(confus_matrix)

              precision    recall  f1-score   support

           0       0.66      0.72      0.69      2147
           1       0.69      0.63      0.66      2140

    accuracy                           0.68      4287
   macro avg       0.68      0.68      0.68      4287
weighted avg       0.68      0.68      0.68      4287

[[1549  598]
 [ 784 1356]]


In [22]:
y_pred = clf.predict(x_train_vector)

classif_report = classification_report(y_train, y_pred)
confus_matrix = confusion_matrix(y_train, y_pred)

print(classif_report)
print(confus_matrix)

              precision    recall  f1-score   support

           0       0.73      0.82      0.77       915
           1       0.80      0.69      0.74       922

    accuracy                           0.76      1837
   macro avg       0.76      0.76      0.76      1837
weighted avg       0.76      0.76      0.76      1837

[[752 163]
 [283 639]]
